# 네이버 블로그 포스팅 주제 분류


네이버 블로그에서는 블로그 주제와 각 포스팅의 주제를 설정할 수 있으나, 이는 블로거가 직접 선택하는 것이기 때문에 실제 주제와 명목상 주제가 다른 경우가 있다. 또한, 여러 주제가 섞여 있는 경우는 블로거 자신도 블로그의 성향과 주제를 알지 못할 때가 있다.

네이버 블로그는 항상 전문성이 있을 수록 좋다. 최근 모집 중인 네이버 인플루언서(또는 이전의 파워블로거)는 전문성이 있어야만 지원할 수 있으며, 포스팅 품질 결정 및 노출 순위 등 여러 부분에서 블로거 전문성이 영향을 미치기 때문이다. 광고주 같은 경우에도 블로그의 주제가 일관된 전문 블로거에게 상품을 제공하는 것이 좋으며, 블로거의 전문성 및 전문 주제를 빠르게 파악할 수 있게 하기 위해 다음과 같은 프로젝트를 진행하게 되었다.

# 데이터 크롤링

In [100]:
import re
import csv
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import Counter
from konlpy.tag import Okt

okt=Okt()

In [ ]:
# 내 블로그 크롤링

f = open("plusotherblog1.csv", "a")
f.write('id'+','+'title'+ ',' + 'contents' + ',' + 'keyword' +'\n')

for i in range (1,180):
    url= "http://blog.naver.com/PostList.nhn?blogId=aquatic_moon&currentPage="+ str(i)
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    blogger = 'aquatic_moon'
    
    hangul=re.compile('[^ ㄱ-ㅣ가-힣]+')
    if (page != None):
        text1=str(soup.find_all("div","se-title-text"))
        text1=re.sub('<.+?>','',text1,0).strip()
        text1=hangul.sub(' ', text1)
        text1=re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ',text1)
        text2=str(soup.find_all("div","se-module se-module-text"))
        text2=re.sub('<.+?>','',text2,0).strip()
        text2=hangul.sub(' ', text2)
        text2=re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ',text2)
        
        nouns=okt.nouns(text2+(text1*10))
        nouns = [n for n in nouns if len(n)>1]

        count = Counter(nouns)
    
        text3 = str(count.most_common(1))
        text3= hangul.sub(' ', text3)
     
   
    f.write(blogger+ ',' + text1 + ',' + text2 + ',' + text3 +'\n')

f.close()

In [ ]:
#타 블로그 크롤링 

f = open("plusotherblog.csv", "a")

blogger = str(input())

for i in range (1,30):
    url= "http://blog.naver.com/PostList.nhn?blogId="+blogger+"&currentPage="+ str(i)
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    hangul=re.compile('[^ ㄱ-ㅣ가-힣]+')
    if (page != None):
        text1=str(soup.find_all("div","se-title-text"))
        text1=re.sub('<.+?>','',text1,0).strip()
        text1=hangul.sub(' ', text1)
        text1=re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ',text1)
        text2=str(soup.find_all("div","se-module se-module-text"))
        text2=re.sub('<.+?>','',text2,0).strip()
        text2=hangul.sub(' ', text2)
        text2=re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ',text2)
        
        nouns=okt.nouns(text2+(text1*10))
        nouns = [n for n in nouns if len(n)>1]

        count = Counter(nouns)
    
        text3 = str(count.most_common(1))
        text3= hangul.sub(' ', text3)
     
   
    f.write(blogger+','+ text1 + ',' + text2 + ',' + text3 +'\n')

f.close()

# 데이터 전처리

<span style="color:red">[ [ [   Do not touch this place   ] ] ]</span>

In [ ]:
# 블로그 포스팅 데이터 불러오기

data = pd.read_csv("plusotherblog.csv", error_bad_lines=False,encoding='CP949')
text = data[['id']+['title']+['contents']]
text.head()

In [ ]:
# 제목(가중치3), 내용(가중치 1)
# okt - 명사만을 남긴 후 저장
# 토큰화 오래 걸리기 때문에 파일 불러와서 사용

text['guel'] = text.apply(lambda row: (okt.nouns(row['title']*3 +row['contents'])), axis=1)
text.head()

text.to_csv("plusotherblog_guel.csv", header = True, index=False, encoding='CP949')

<span style="color:red">[ [ [   Do not touch this place   ] ] ]</span>

----

In [101]:
# 단어 토큰화 된 'guel'만 가져오기

data = pd.read_csv("plusotherblog_guel.csv", error_bad_lines=False,encoding='CP949')
text = data[['guel']]

text.head()

,guel
0,"['년', '월', '월', '공모전', '추천', '경기도', '코로나', '극복..."
1,"['별', '꽃', '구름', '이모티콘', '모음', '표정', '특수문자', '..."
2,"['표정', '이모티콘', '특수문자', '이모지', '모음', '표정', '이모티..."
3,"['표정', '이모티콘', '특수문자', '이모지', '모음', '표정', '이모티..."
4,"['아이폰', '화면', '녹화', '소리', '안', '때', '마이크', '녹음..."


In [102]:
# blogger id만 추출

idset = set()
for index, value in enumerate(data['id']):
     idset.add(value)

In [103]:
blog_dict={"game":["12512512","nemonangoom","doek11","jina3421","puck1001","meniereman","anjffh09"],
           "car": ['ohjunkwon','gmgeo','acartunning','cane_h','cogram',	'steg1011','pagani0101'],
           "animation": ['tjdmsgml8099',	'kschle5841',	'rladnjstjd55',	'sangyup1028',	'bakemono1',	'897jos'],
           "travel":['neweunha',	'dufwjddlftkd',	'tnwlsdl702',	'hyun68',	'coreabis',	'sunae628',	'khjins5918',	'cutetnwls',	'feny'],
           "beauty":['chs3202',	'siehl',	'wlgysnl23',	'yangeunb',	'neriia',	'iamchocolat',	'youridalbit',	'gyerimong2',	'dlcmalsis',	'clarisecho',	'hello_phe',	'dameebaby'],
           "drama":['film1982',	'ddazero',	'imaplus',	'js2y86'],
           "sports":['givehyun',	'mittlivsom',	'zkskftl',	'huck15',	'madden789',	'jgjj7116crazy828282',	'z2inny',	'evenfriend',	'nwoongwin',	'yelin1890',	'dkfnal2232',],
           "pet":['lbygns12',	'choi540',	'dean4001',	'lbygns123',	'sardonic77',	'hyovelyvov'],
           "movie":['luckymasiwoo',	'gomsolvie',	'shysung28',	'rnjsekgml052',	'forsea5999',	'cay2011',	'above_cloud',	'casachampion',	'sgnam10',	'ldi25'],
           "delicious":['fixion84',	'visavisu',	'meal8030',	'dmsgur9122',	'clala70',	'saypumpy',	'darkdurki',	'ssyj963',	'hhhrrr33',	'bongja814',	'dannyelf',	'kyakya1633'],
           "baby":['dnjfgid2000',	'kaen17',	'andyfirstlv',	'taehun0930',	'hjamy131'],
           "cook":['king700203',	'mildek',	'mijinkim80',	'kimhy004',	'beluga11',	'juhyun9909',	'yoonjs3',	'gugu9416',	'77huiya',],
           "music":['mal3500','ksy1327'],
           "fashion":['purplemodern',	'sdbuyer',	'jayuyu',	'bushcherry1',],
           "book":['didini',	'sandhya',	'ohjtolstoy',	'next6156',	'lotusksm',],
           "DIY":['i_pattern',	'iqfull01',	'luna926',	'nalda21',],
           "IT":['winsweet',	'moimoi1357',	'seagreen0314',	'mr_choo',	'moyeu119',	'tech-plus',	'neces2',	'soary81',	'aiwositai',	'lucky_box7',],
           "cash":['sirabon',	'ninohaeram',	'resumet',	'wmftkah',	'jeunkim',	'chang0333',	'rmsgud2007',	'hwre7774',	'stockclinic',]
          }

In [104]:
# 토큰화된 데이터 -> string 형식으로 불러오는 문제발생
# 특수문자 제거 후 단어 재 strip

text = text.replace('\'','',regex=True)
text = text.replace('\[','',regex=True)
text = text.replace(',','',regex=True)
text = text.replace('\]','',regex=True)
text['guel'].str.strip()

text.head()

<>:5: DeprecationWarning: invalid escape sequence \[
<>:7: DeprecationWarning: invalid escape sequence \]
<>:5: DeprecationWarning: invalid escape sequence \[
<>:7: DeprecationWarning: invalid escape sequence \]
<>:5: DeprecationWarning: invalid escape sequence \[
<>:7: DeprecationWarning: invalid escape sequence \]
<ipython-input-104-563d76bd41e0>:5: DeprecationWarning: invalid escape sequence \[
  text = text.replace('\[','',regex=True)
<ipython-input-104-563d76bd41e0>:7: DeprecationWarning: invalid escape sequence \]
  text = text.replace('\]','',regex=True)


,guel
0,년 월 월 공모전 추천 경기도 코로나 극복 위 영상 콘텐츠 공모전 년 월 월 공모전...
1,별 꽃 구름 이모티콘 모음 표정 특수문자 별 꽃 구름 이모티콘 모음 표정 특수문자 ...
2,표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
3,표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
4,아이폰 화면 녹화 소리 안 때 마이크 녹음 방법 아이폰 화면 녹화 소리 안 때 마이...


In [105]:
# tokenize 를 위한 문서화

text['guel'] = text.apply(lambda row: (row['guel']), axis=1)
text['guel'].head()

0    년 월 월 공모전 추천 경기도 코로나 극복 위 영상 콘텐츠 공모전 년 월 월 공모전...
1    별 꽃 구름 이모티콘 모음 표정 특수문자 별 꽃 구름 이모티콘 모음 표정 특수문자 ...
2    표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
3    표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
4    아이폰 화면 녹화 소리 안 때 마이크 녹음 방법 아이폰 화면 녹화 소리 안 때 마이...
Name: guel, dtype: object

In [106]:
# tokenize

import nltk
text['guel']= text.apply(lambda row: nltk.word_tokenize (row['guel']), axis=1)
text.head()

,guel
0,"[년, 월, 월, 공모전, 추천, 경기도, 코로나, 극복, 위, 영상, 콘텐츠, 공..."
1,"[별, 꽃, 구름, 이모티콘, 모음, 표정, 특수문자, 별, 꽃, 구름, 이모티콘,..."
2,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,..."
3,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,..."
4,"[아이폰, 화면, 녹화, 소리, 안, 때, 마이크, 녹음, 방법, 아이폰, 화면, ..."


In [107]:
# 불용어 제거

temp=[]

with open('./stop_words.txt', 'r',encoding='utf-8') as stop:
    lines = stop.readlines()
    for line in lines:
        line=re.sub('\n','',line,0).strip()
        temp.append(line)

stop_words=' '.join(map(str, temp))
stop_words=stop_words.split(' ')

text['stopwords']=text['guel'].apply(lambda x: [word for word in x if word not in stop_words])
text.head()

,guel,stopwords
0,"[년, 월, 월, 공모전, 추천, 경기도, 코로나, 극복, 위, 영상, 콘텐츠, 공...","[년, 월, 월, 공모전, 경기도, 코로나, 극복, 위, 영상, 콘텐츠, 공모전, ..."
1,"[별, 꽃, 구름, 이모티콘, 모음, 표정, 특수문자, 별, 꽃, 구름, 이모티콘,...","[별, 꽃, 구름, 이모티콘, 모음, 표정, 특수문자, 별, 꽃, 구름, 이모티콘,..."
2,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...","[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,..."
3,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...","[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,..."
4,"[아이폰, 화면, 녹화, 소리, 안, 때, 마이크, 녹음, 방법, 아이폰, 화면, ...","[아이폰, 화면, 녹화, 소리, 안, 때, 마이크, 녹음, 아이폰, 화면, 녹화, ..."


In [108]:
# 길이 1인 단어 삭제

tokenized_doc = text['stopwords'].apply(lambda x : [word for word in x if len(word) > 1])
tokenized_doc[:5]

0    [공모전, 경기도, 코로나, 극복, 영상, 콘텐츠, 공모전, 공모전, 경기도, 코로...
1    [구름, 이모티콘, 모음, 표정, 특수문자, 구름, 이모티콘, 모음, 표정, 특수문...
2    [표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...
3    [표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...
4    [아이폰, 화면, 녹화, 소리, 마이크, 녹음, 아이폰, 화면, 녹화, 소리, 마이...
Name: stopwords, dtype: object

In [109]:
#역토큰화

detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
text['stopwords'] = detokenized_doc

text.to_csv("plusotherblog_dt.csv", header = True, index=False, encoding='CP949')
text.head()

,guel,stopwords
0,"[년, 월, 월, 공모전, 추천, 경기도, 코로나, 극복, 위, 영상, 콘텐츠, 공...",공모전 경기도 코로나 극복 영상 콘텐츠 공모전 공모전 경기도 코로나 극복 영상 콘텐...
1,"[별, 꽃, 구름, 이모티콘, 모음, 표정, 특수문자, 별, 꽃, 구름, 이모티콘,...",구름 이모티콘 모음 표정 특수문자 구름 이모티콘 모음 표정 특수문자 구름 이모티콘 ...
2,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...",표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
3,"[표정, 이모티콘, 특수문자, 이모지, 모음, 표정, 이모티콘, 특수문자, 이모지,...",표정 이모티콘 특수문자 이모지 모음 표정 이모티콘 특수문자 이모지 모음 표정 이모티...
4,"[아이폰, 화면, 녹화, 소리, 안, 때, 마이크, 녹음, 방법, 아이폰, 화면, ...",아이폰 화면 녹화 소리 마이크 녹음 아이폰 화면 녹화 소리 마이크 녹음 아이폰 화면...


In [110]:
# 문서 내 단어 Count
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorize = CountVectorizer(min_df=2)
X = vectorize.fit_transform(text['stopwords'])
print('(sentence {}, feature {})'.format(X.shape[0], X.shape[1]))

array = pd.DataFrame(X.toarray())
array

(sentence 4021, feature 32814)


,0,1,2,3,4,5,6,7,8,9,...,32804,32805,32806,32807,32808,32809,32810,32811,32812,32813
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
# Tfid Vector
vectorizer = TfidfVectorizer(stop_words=None, max_features=1400)
Y = vectorizer.fit_transform(text['stopwords'])

Y.shape
array1 = pd.DataFrame(Y.toarray())
array1

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.066729,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.149361,...,0.000000,0.0,0.0,0.0,0.0,0.141101,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4018,0.0,0.0,0.0,0.0,0.033033,0.0,0.0,0.0,0.0,0.000000,...,0.024024,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4019,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


# LDA 잠재 디리클레 할당 (Latent Dirichlet Allocation)

  
* 토픽모델링: 문서 집합에서 토픽 찾아내는 프로세스
* LDA : 단어가 특정 토픽에 존재할 활률과 문서에 특정 토픽 존재할 확률을 결합 확률로 추정해서 토픽 추출  


- 토픽 개수 사용자가 정하는 것
- 모든 단어를 k개 중 하나의 토픽에 할당
- 모든 문서의 모든 단어에 아래 사항 반복  


- 문서 내 단어들이 어떤 토픽인지, 첫번째기준은 문서내 단어들이 많이 할당된 토픽으로
- 모든 문서 중 해당 단어 어떤 토픽 많이 할당 되었는지


In [112]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=22, learning_method='online', random_state=777, max_iter=1)
lda_top = lda_model.fit_transform(Y)
print(lda_model.components_.shape)

(22, 1400)


In [113]:
# 단어사전
terms = vectorizer.get_feature_names() 

In [114]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [115]:
# Topic 별 최빈 단어

import gensim

NUM_TOPICS = 22 #22개의 토픽, k=22
%time ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, random_state=9)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Wall time: 6min 9s
(7, '0.014*"맛집" + 0.014*"카페" + 0.010*"주문" + 0.008*"여행"')
(20, '0.029*"듀얼" + 0.026*"플레이" + 0.023*"서버" + 0.023*"센스"')
(0, '0.025*"호텔" + 0.010*"분유" + 0.009*"그린" + 0.009*"오일"')
(17, '0.048*"피부" + 0.012*"쿠션" + 0.011*"수분" + 0.011*"케어"')
(4, '0.030*"컬러" + 0.016*"메이크업" + 0.015*"팔레트" + 0.014*"섀도우"')
(11, '0.015*"투자" + 0.012*"시장" + 0.011*"종목" + 0.010*"주식"')
(13, '0.012*"노래" + 0.009*"활동" + 0.008*"앨범" + 0.007*"사랑"')
(6, '0.071*"컬러" + 0.017*"렌즈" + 0.014*"핑크" + 0.014*"레드"')
(5, '0.011*"건강" + 0.008*"섭취" + 0.007*"선물" + 0.006*"관리"')
(10, '0.012*"노트북" + 0.012*"작업" + 0.011*"모니터" + 0.011*"화면"')
(8, '0.016*"강아지" + 0.016*"시즌" + 0.007*"드라마" + 0.007*"반려견"')
(1, '0.015*"벤큐" + 0.014*"재료" + 0.013*"레시피" + 0.012*"요리"')
(18, '0.019*"원피스" + 0.017*"디자인" + 0.013*"스타" + 0.012*"가방"')
(2, '0.035*"영화" + 0.011*"작품" + 0.006*"사랑" + 0.005*"배우"')
(14, '0.058*"청소" + 0.038*"무선" + 0.037*"먼지" + 0.027*"선풍기"')
(19, '0.026*"게임" + 0.016*"이벤트" + 0.014*"기가바이트" + 0.014*"캐릭터"')
(16, '0.058*"고양이" + 0.039*"마사지" + 0.026*"모

In [116]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(vis,'LDA.html')
pyLDAvis.display(vis)

In [117]:
#내재적 평가: 결과 사람이 판단하기에 적합?
#Coherence (내재적 평가) - 토픽 일관성

from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_doc, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('>>> Coherence Score: ', coherence_lda)
print('>>> Perplexity: ', ldamodel.log_perplexity(corpus))

>>> Coherence Score:  0.6198143459772262
>>> Perplexity:  -9.189978818649895


In [118]:
# 문서별 주제 보기

def doc_topic(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key = lambda x: (x[1]), reverse = True)
        
        for j,(topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                topic_table = topic_table.append(pd.Series([int(topic_num),round(prop_topic,4), topic_list]), ignore_index=True)
                
            else:
                break
    return(topic_table)

topictable =  doc_topic(ldamodel,corpus)
topictable = topictable.reset_index()

topictable.columns = ['index', 'topic','most common','every topic']
topictable[100:120]

,index,topic,most common,every topic
100,100,13.0,0.6820,"[(0, 0.14528124), (2, 0.07033981), (6, 0.01911..."
101,101,7.0,0.7636,"[(1, 0.22824807), (7, 0.7635558)]"
102,102,7.0,0.8597,"[(4, 0.025754897), (7, 0.8597046), (19, 0.0284..."
103,103,7.0,0.7635,"[(0, 0.013420884), (1, 0.21488912), (7, 0.7634..."
104,104,7.0,0.7259,"[(1, 0.26650384), (7, 0.7259163)]"
105,105,7.0,0.5689,"[(4, 0.07121412), (5, 0.013464588), (6, 0.1143..."
106,106,7.0,0.9182,"[(0, 0.06103996), (2, 0.012376527), (7, 0.9181..."
107,107,7.0,0.6061,"[(0, 0.06834651), (5, 0.30997127), (7, 0.60612..."
108,108,7.0,0.7986,"[(1, 0.030124383), (5, 0.15369835), (7, 0.7986..."
109,109,13.0,0.7804,"[(5, 0.020651588), (8, 0.025453124), (12, 0.14..."


In [119]:
# topic table과 blogger 정보 table 연결

df1= topictable[['index']]
df2= data[['id']+['title']]
df3=topictable[['topic']+['most common']+['every topic']]
df4= pd.concat([df1, df2, df3], axis=1)
df4[105:125]

,index,id,title,topic,most common,every topic
105,105,aquatic_moon,나도 모르게 돈 쓰고 나오는 강릉 소품샵 강릉문방구,7.0,0.5689,"[(4, 0.07121412), (5, 0.013464588), (6, 0.1143..."
106,106,aquatic_moon,예쁜 앤틱 가구가 가득한 강릉시내카페 년임당방앗간,7.0,0.9182,"[(0, 0.06103996), (2, 0.012376527), (7, 0.9181..."
107,107,aquatic_moon,동탄 신도시 홍익돈까스 이게 어떻게 인분인가요,7.0,0.6061,"[(0, 0.06834651), (5, 0.30997127), (7, 0.60612..."
108,108,aquatic_moon,영통 초밥 맛집 주원초밥 앤 참치,7.0,0.7986,"[(1, 0.030124383), (5, 0.15369835), (7, 0.7986..."
109,109,aquatic_moon,음악추천 꼭 챙겨 듣는 세븐틴 노래 수록곡 중심으로,13.0,0.7804,"[(5, 0.020651588), (8, 0.025453124), (12, 0.14..."
110,110,aquatic_moon,대충 살자 야식으로 간장밥 볶아 먹는 나처럼,1.0,0.8518,"[(0, 0.033194263), (1, 0.851801), (7, 0.030926..."
111,111,aquatic_moon,권선동미용실 감동을 전하다 에서 클리닉하다,17.0,0.3367,"[(2, 0.09772273), (7, 0.3064489), (17, 0.33668..."
112,112,aquatic_moon,수원 인계동 시 맛집 수원시청역 북촌손만두 피냉면 먹었어요,7.0,0.6712,"[(1, 0.13241404), (7, 0.6712342), (10, 0.02238..."
113,113,aquatic_moon,일상 월의 기억 홍콩 마카오 여행 정리하기 센트럴 구경하기,7.0,0.5224,"[(0, 0.3467875), (4, 0.036276355), (6, 0.04465..."
114,114,aquatic_moon,아이패드 블루투스 키보드 퓨전에프앤씨 아이노트 후기,21.0,0.3316,"[(4, 0.04651429), (5, 0.011759468), (9, 0.2186..."


In [120]:
def get_key(val):
    for key, value in blog_dict.items(): 
         if val in value: 
            return key

In [121]:
topic_dict={0.0:"travel", 1.0:"cook" , 2.0:["drama","movie"], 3.0:"car", 4.0:["beauty","fashion"], 
    5.0:"other", 6.0:["beauty","fashion"], 7.0:"delicious", 8.0:["pet","baby"], 9.0:"IT", 10.0:"IT",
    11.0:"cash", 12.0:"travel", 13.0:"music", 14.0:"other", 15.0:"sports", 16.0:["pet","baby"], 
    17.0:["beauty","fashion"], 18.0:["beauty","fashion"], 19.0:"game",20.0:"game", 21.0:["pet","baby"]}

In [122]:
# LDA로 추정한 블로그 주제

blog_topic = pd.DataFrame()
grade=0
grade2=0

for blogger in idset:
        df5=df4.loc[df4['id']==blogger,['id','topic']]
        counts=df5['topic'].value_counts().to_dict()
        dic_max=max(counts, key=counts.get)
        
        if str(get_key(blogger)) in  topic_dict.get(dic_max) :
            grade+=1
            check = "True"
            
        else:
            check = "False"
        
        if str(get_key(blogger)) in ["DIY","book","animation"]:
            grade2+=1
            
        blog_topic = blog_topic.append(pd.Series([blogger, dic_max, topic_dict.get(dic_max), get_key(blogger), check]), ignore_index=True)
        
        
blog_topic.columns = ['blogger', 'topic_num', 'topic', 'real_topic','check']


print(">>> other 추가 정답률:{} / {}  =============== {}%".format(grade-1,len(idset)-1, (grade-1)/(len(idset)-1)*100))
print(">>> other 제외 정답률:{} / {}  =============== {}%".format(grade-1,len(idset)-grade2-1, (grade-1)/(len(idset)-grade2-1)*100))
blog_topic[80:100]

>>> other 추가 정답률:86 / 132  =============== 65.15151515151516%
>>> other 제외 정답률:86 / 117  =============== 73.50427350427351%


,blogger,topic_num,topic,real_topic,check
80,nwoongwin,21.0,"[pet, baby]",sports,False
81,luckymasiwoo,2.0,"[drama, movie]",movie,True
82,resumet,11.0,cash,cash,True
83,andyfirstlv,21.0,"[pet, baby]",baby,True
84,doek11,19.0,game,game,True
85,ksy1327,13.0,music,music,True
86,kschle5841,12.0,travel,animation,False
87,dean4001,16.0,"[pet, baby]",pet,True
88,dannyelf,7.0,delicious,delicious,True
89,yelin1890,21.0,"[pet, baby]",sports,False


In [123]:
#LDA로 결정된 나의 블로그 주제

df5=df4.loc[df4['id']=="aquatic_moon",['id','topic']]

counts=df5['topic'].value_counts().to_dict()
print(counts)

dic_max=max(counts, key=counts.get)
print(dic_max,'\n')
print(">>> My blog topic : " ,topic_dict.get(dic_max))

{7.0: 71, 21.0: 19, 8.0: 16, 17.0: 11, 13.0: 10, 1.0: 8, 2.0: 7, 5.0: 6, 10.0: 5, 0.0: 5, 9.0: 4, 12.0: 3, 18.0: 2, 3.0: 2, 16.0: 2, 4.0: 2, 14.0: 2, 11.0: 2, 6.0: 1, 19.0: 1}
7.0 

>>> My blog topic :  delicious


# K-means

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import pyLDAvis
from kmeans_to_pyLDAvis import kmeans_to_prepared_data
from soyclustering import SphericalKMeans

In [125]:
naverData = pd.read_csv('plusotherblog_dt.csv',encoding='CP949')
docs = naverData['stopwords'].values.tolist()

In [126]:
# Count vector -> L2 정규화 (단어 문서 유클리드 거리)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(naverData['stopwords'].values.astype('U'))
X = normalize(X, norm='l2')

In [127]:
# Kmeans

%time kmeans = KMeans(n_clusters=100,random_state=0).fit(X)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

Wall time: 19min 27s


In [136]:
# 시각화

vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]

prepared_data = kmeans_to_prepared_data(
    X, vocab, centers, labels,
    embedding_method='tsne')

pyLDAvis.display(prepared_data)

In [129]:
# topic labeling

def get_key2(val): 
    for key, value in topic_dict5.items(): 
         if val in value: 
            return key
        

topic_dict5={"baby":(19,67,92,95),"pet":(15,23,28,58,66), "travel":(17,74,77,88,94), "beauty":(2,9,16,32,65,71,81),
            "fashion":(36,40), "sports":(8,10,29,54,55,64,68), "delicious":(6,13,18,33,37,48,70,76,79,83,84,89),
            "cook":(3,69,85,97), "IT":(4,5,38,41,50,56,57,86,98,99), "drama":(11,61,72,78), "movie": (34,49,90),
            "car":(7,35,43,62), "cash": (1,24,26,46,51,59,60,80,82), "game": (25,30,73,96), "music":(39,42,52),
            "animation":(0,75), "book": (22,93), "DIY":(91,), "other":(12,14,20,21,27,31,44,45,47,53,63,87)}

In [130]:
data = pd.read_csv("plusotherblog.csv", error_bad_lines=False,encoding='CP949')
text = data[['id']+['title']+['contents']]
text.head()

df= pd.DataFrame()
df['result']= labels

result =[]

for label in labels:
    result.append(get_key2(label))

df['topic']= result

df = pd.concat([data['id'].reset_index(drop=True), data['title'].reset_index(drop=True), df],axis=1)
df[105:120]

,id,title,result,topic
105,aquatic_moon,나도 모르게 돈 쓰고 나오는 강릉 소품샵 강릉문방구,13,delicious
106,aquatic_moon,예쁜 앤틱 가구가 가득한 강릉시내카페 년임당방앗간,13,delicious
107,aquatic_moon,동탄 신도시 홍익돈까스 이게 어떻게 인분인가요,33,delicious
108,aquatic_moon,영통 초밥 맛집 주원초밥 앤 참치,18,delicious
109,aquatic_moon,음악추천 꼭 챙겨 듣는 세븐틴 노래 수록곡 중심으로,52,music
110,aquatic_moon,대충 살자 야식으로 간장밥 볶아 먹는 나처럼,97,cook
111,aquatic_moon,권선동미용실 감동을 전하다 에서 클리닉하다,32,beauty
112,aquatic_moon,수원 인계동 시 맛집 수원시청역 북촌손만두 피냉면 먹었어요,18,delicious
113,aquatic_moon,일상 월의 기억 홍콩 마카오 여행 정리하기 센트럴 구경하기,74,travel
114,aquatic_moon,아이패드 블루투스 키보드 퓨전에프앤씨 아이노트 후기,38,IT


In [131]:
new_blog_topic = pd.DataFrame()

grade=0

for blogger in idset:
        df2=df.loc[df['id']==blogger, ['id','title','result','topic']]
        counts=df2['topic'].value_counts().to_dict()
        dic_max=max(counts, key=counts.get)
        
        if dic_max == get_key(blogger):
            check="True"
            grade+=1
            
        else:
            check="False"
            
        new_blog_topic = new_blog_topic.append(pd.Series([blogger, dic_max,get_key(blogger), check]), ignore_index=True)
        
        
        
new_blog_topic.columns = ['blogger', 'topic', 'real_topic','check']
        

print(">>> other 추가 정답률:{} / {}  =============== {}%".format(grade-1,len(idset)-1, (grade-1)/(len(idset)-1)*100))
new_blog_topic[65:85]

>>> other 추가 정답률:98 / 132  =============== 74.24242424242425%


,blogger,topic,real_topic,check
65,anjffh09,game,game,True
66,tjdmsgml8099,animation,animation,True
67,sangyup1028,other,animation,False
68,dameebaby,beauty,beauty,True
69,hhhrrr33,delicious,delicious,True
70,pagani0101,car,car,True
71,jgjj7116,other,None,False
72,cane_h,car,car,True
73,mal3500,music,music,True
74,feny,travel,travel,True


In [132]:
# 내 블로그 주제

my = (df['id']=='aquatic_moon') & (df['topic']!='other')
my_df=df[my]
counts=my_df['topic'].value_counts().to_dict()
dic_max=max(counts, key=counts.get)
print(counts,'\n')
print(">>> My blog topic : " ,dic_max)

{'delicious': 52, 'IT': 21, 'travel': 10, 'music': 8, 'beauty': 7, 'drama': 4, 'cook': 3, 'movie': 2, 'car': 1, 'game': 1, 'pet': 1} 

>>> My blog topic :  delicious


In [133]:
from __future__ import print_function
from gensim.models import KeyedVectors
ko_model = KeyedVectors.load_word2vec_format('wiki.ko.vec')

In [134]:
data = pd.DataFrame(prepared_data.token_table)

k=1
all = 0
co=0

excepttopic=0 #군집크기가 10안되는 작은 군집 고려 안함

while(k<=100):
    temp=0
    exceptnum=0
    
    filter = data[data['Topic'] == k]
    
    if (len(filter) > 5):
        print("-------------------")
    else:
        print("-------------------")
        excepttopic+=1
        pass
        
    word = pd.DataFrame(filter['Term'])
    word= pd.DataFrame(word.reset_index()['Term'])
    
    for i in range(5):
        for j in range(5):
            try:
                term1= (''.join(map(str,word.values[i])))
                term2= (''.join(map(str, word.values[j])))
                sim_score = ko_model.wv.similarity(term1, term2)
            except:
                exceptnum+=1
                pass
            
            temp+=(sim_score)
            
            
    temp= temp-5+exceptnum
    co= ((temp)/((5-exceptnum)*(4-exceptnum))) #'덕질'과 같은 vec에 없는 단어 아예 고려 안함
    print(k,"번 Cluster의 Word2Vec Similarity : ", co)
    
    
    if k in [13,15,21,22,28,32,45,46,48,54,64,88]:
        excepttopic+=1
        co=0
        print(co)
        
    all+=co
    k+=1

all= all/(100-excepttopic)


-------------------
1 번 Cluster의 Word2Vec Similarity :  0.3900091782212257
-------------------


C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call t

2 번 Cluster의 Word2Vec Similarity :  1.0461661398410798
-------------------
3 번 Cluster의 Word2Vec Similarity :  0.44759810268878936
-------------------
4 번 Cluster의 Word2Vec Similarity :  0.5321110516786576
-------------------
5 번 Cluster의 Word2Vec Similarity :  0.42834845185279846
-------------------


C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call t

6 번 Cluster의 Word2Vec Similarity :  0.4255869358778
-------------------
7 번 Cluster의 Word2Vec Similarity :  0.8128939867019653
-------------------
8 번 Cluster의 Word2Vec Similarity :  0.36296417713165285
-------------------
9 번 Cluster의 Word2Vec Similarity :  0.4743494302034378
-------------------


...

C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call t

92 번 Cluster의 Word2Vec Similarity :  0.49894979000091555
-------------------
93 번 Cluster의 Word2Vec Similarity :  0.27205381244421006
-------------------
94 번 Cluster의 Word2Vec Similarity :  0.3154522642493248
-------------------


C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call t

95 번 Cluster의 Word2Vec Similarity :  0.22601269595324994
-------------------
96 번 Cluster의 Word2Vec Similarity :  0.32830403447151185
-------------------
97 번 Cluster의 Word2Vec Similarity :  0.4395640045404434
-------------------
98 번 Cluster의 Word2Vec Similarity :  0.48695110380649564
-------------------
99 번 Cluster의 Word2Vec Similarity :  0.5185354351997375
-------------------
100 번 Cluster의 Word2Vec Similarity :  0.2555205121636391


C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call t

In [135]:
print(">>> Kmeans Word2Vec similarity average : ", all)
print(">>> 포함되지 않은 Clustering 개수 : " ,excepttopic)

>>> Kmeans Word2Vec similarity average :  0.4298701860909833
>>> 포함되지 않은 Clustering 개수 :  13


In [182]:
#Similarity 40% 예시

print(ko_model.wv.similarity("음악","가사"))
print(ko_model.wv.similarity("영화","넷플릭스"))
print(ko_model.wv.similarity("비행기","여행"))
print(ko_model.wv.similarity("파운데이션","립스틱"))

0.40727895
0.48753843
0.4325611
0.41064075


C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
C:\Users\Hyennnnnnnnnnnn\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
